The purpose of this notebook is mainly to check the correctness of new real values spherical harmonics


In [1]:
import sys
sys.path.insert(0, '../')
import numpy as np
import ase.io as ase_io
from ase import Atoms

import pyximport; pyximport.install()
import nice_utilities
import naive, radial_basis, spherical_coefficients, spherical_harmonics, convert_rascal_coefficients
from convert_rascal_coefficients import convert_rascal_coefficients, normalize_by_ps
import ClebschGordan, test_utilities

/home/pozdn/.local/lib/python3.6/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/pozdn/methane_nice/collecting_standard_features/cleaned/convert_rascal_coefficients.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [2]:
import rascal
from rascal.representations import SphericalInvariants as SOAP
from rascal.representations import SphericalExpansion as SPH
from rascal.neighbourlist.structure_manager import (
        mask_center_atoms_by_species, mask_center_atoms_by_id)

In [3]:
def get_rascal_coefficients(structures, r_cut, n_max, l_max, n_types, normalize = True):
    HYPERS = {
    'interaction_cutoff': r_cut,
    'max_radial': n_max,
    'max_angular': l_max,
    #'gaussian_sigma_constant': 0.05,
    'gaussian_sigma_type': 'Constant',
    'cutoff_smooth_width': 0.3,
    'radial_basis': 'GTO',
    #'expansion_by_species_method' : 'environment wise'
    }
    
    nat=[]
    y=[]
    for structure in structures: 
        structure.cell =[120, 120, 120]
        structure.positions += np.asarray((60,60,60))
        structure.pbc=True
        nat.append(5)
        structure.wrap()
    nat = np.asarray(nat)
    n_envs = nat.sum()
   
    
    sph = SPH(**HYPERS)
    feat = sph.transform(structures).get_features(sph)
    res = convert_rascal_coefficients(feat, n_max, n_types, l_max)
    
    if (normalize):
        normalize_by_ps(res)
    return np.array(res)

In [4]:
structure = ase_io.read('../structures.xyz', index = '0:1')[0]

In [5]:
env = structure.positions[1:]
print(env)

[[ 1.1957718   0.69582632 -1.5377768 ]
 [-0.07590599  1.04620262  1.95058034]
 [-1.10813614  1.91728119  1.81155538]
 [ 1.39764218 -0.98903322 -0.05350022]]


In [6]:
coefs = get_rascal_coefficients([structure], 6.3, 5, 5, 2, normalize = True)
print(coefs.shape)
coefs = coefs[0]
print(coefs.shape)

(5, 10, 6, 11)
(10, 6, 11)


In [7]:
ps = np.array(naive.compute_powerspectrum(coefs, 5))
print(ps.shape)
print(np.sum(np.abs(ps)))
print(np.sum(ps * ps))
print(ps[0:10])

(330,)
5.035966559525711
1.0000000000000004
[0.01122829 0.01681375 0.01668307 0.02553262 0.0266217  0.02523632
 0.04755694 0.03686422 0.03286326 0.08741215]


In [8]:
for _ in range(10):
    rotated = Atoms(structure)
    rotated.positions = test_utilities.rotate_env(structure.positions)
    coefs_rotated = get_rascal_coefficients([rotated], 6.3, 5, 5, 2)[0]
    ps_new = naive.compute_powerspectrum(coefs_rotated, 5)
    print(np.sum(np.abs(ps - ps_new)))

7.822681220105493e-14
4.966014748128845e-14
6.685064133526595e-14
9.156143928440962e-14
5.69159951848276e-14
6.625172890221791e-14
8.348958968563882e-14
5.694156880357111e-14
1.0019392728547382e-13
7.833846639009609e-14


In [9]:
clebsch = ClebschGordan.ClebschGordan(5)

In [10]:
bs = np.array(naive.compute_bispectrum(clebsch.precomputed, coefs, 5, False))

print(bs.shape)
print(np.sum(np.abs(bs)))
print(np.sum(bs * bs))
print(bs[0:20])

(24420,)
16.737135598915504
1.2313432094855783
[ 1.68261838e-03  2.51962885e-03  2.50004579e-03  3.82619643e-03
  3.98940103e-03  3.78179474e-03  2.51962885e-03 -1.45470839e-03
  0.00000000e+00  2.30623460e-03 -1.78640164e-03  6.77626358e-21
  2.46548177e-03 -2.08371241e-03 -2.71050543e-20  3.16518514e-03
 -2.79143091e-03 -2.71050543e-20  3.26152950e-03 -2.95016443e-03]


In [11]:
for _ in range(10):
    bs = naive.compute_bispectrum(clebsch.precomputed, coefs, 5, False)
    bs = np.array(bs)
    rotated = Atoms(structure)
    rotated.positions = test_utilities.rotate_env(structure.positions)
    coefs_rotated = get_rascal_coefficients([rotated], 6.3, 5, 5, 2)[0]
   
    bs_new = naive.compute_bispectrum(clebsch.precomputed, coefs_rotated, 5, False)
    bs_new = np.array(bs_new)
    print(np.sum(np.abs(bs - bs_new)))
    

7.494982072376525e-13
6.47394999817382e-13
5.223973906134527e-13
3.68859935186583e-13
3.8122878521148325e-13
8.004296983315355e-13
8.948619329229781e-13
6.065529424280475e-13
3.3378328965133683e-13
4.1213026903759484e-13


In [12]:
print(coefs[0][2])

[-0.10492458 -0.00221463 -0.04264645  0.04169768  0.04595199  0.
  0.          0.          0.          0.          0.        ]


In [13]:
print(coefs_rotated[0][2])

[ 0.02404831  0.09207597  0.03848061  0.06878598 -0.03760985  0.
  0.          0.          0.          0.          0.        ]
